# Estimate the costs for a SMOS Level 2C Datacube

**CAUTION:** The following cell must be identical to the corresponding one in `notebook.ipynb`.

In [1]:
product_type: str = "SMOS-L2C-SM"   # The product type. Either "SMOS-L2C-SM" or "SMOS-L2C-OS".
time_range: str = "2022-01-01/2022-01-03" # The time range using format "<start>/<end>".
agg_interval: str = "1d"  # Aggregation intervall
res_level: int = 0  # Spatial resolution level in the range 0 to 4. Zero refers to max.

In [2]:
import scrapbook
import pandas as pd
from helpers import get_time_ranges

In [3]:
time_ranges = get_time_ranges(time_range, agg_interval)

# There are ~29 SMOS product files per day
num_input_files = 29 * len(time_ranges) 

if agg_interval:
    start = time_ranges[0][0]
    stop = time_ranges[1][1]
    num_output_steps = int((pd.Timestamp(stop) - pd.Timestamp(start)) 
                           / pd.Timedelta(agg_interval))
else:
    num_output_steps = num_input_files

# Max image output resolution in pixels
max_x = 8192
max_y = 4032

num_output_pixels = (max_x * max_y) / 2 ** res_level

num_output_voxels = num_output_pixels * num_output_steps

In [4]:
def compute_price(num_input_files: int, num_output_voxels: int) -> int:
    a = 1.0   # cost of an input SMOS file
    b = 1e-6  # cost of an output datacube voxel
    return a * num_input_files + b * num_output_voxels

In [5]:
price = compute_price(num_input_files, num_output_voxels)
price

120.030144

In [6]:
# Transmit the price to the EDC platform
scrapbook.glue("", price)